# Upsampling Tutorial


## Worked Example Using the UpSampling2D Layer

In [1]:
# example of using the upsampling layer
from numpy import asarray
from keras.models import Sequential
from keras.layers import UpSampling2D

In [2]:
# define input data
X = asarray([[1, 2],
			 [3, 4]])

# show input data for context
print(X)

[[1 2]
 [3 4]]


In [3]:
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

The model has only the UpSampling2D layer which takes 2×2 grayscale images as input & outputs the result of the upsampling operation.

In [4]:
# define model
model = Sequential()
model.add(UpSampling2D(input_shape=(2, 2, 1)))

# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 4, 4, 1)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [5]:
# make a prediction with the model - upsample a provided input image
yhat = model.predict(X)

In [6]:
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))

# summarize output
print(yhat)

# We can see that it will output a 4×4 result as we expect, and importantly, the layer has no parameters or model weights. 
# This is because it is not learning anything; it is just doubling each row and column for our input data.

[[1. 1. 2. 2.]
 [1. 1. 2. 2.]
 [3. 3. 4. 4.]
 [3. 3. 4. 4.]]


## Simple Generator Model With the UpSampling2D Layer

The UpSampling2D layer is not able to fill in useful detail in the upsampling operation. To be useful in a GAN, each UpSampling2D layer must be followed by a Conv2D layer that will learn to interpret the doubled input and be trained to translate it into meaningful detail.

In this example, our little GAN generator model must produce a 10×10 image and take a 100 element vector from the latent space as input.

In [7]:
# example of using upsampling in a simple generator model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Conv2D


A Dense fully connected layer can be used to interpret the input vector and create a sufficient number of activations (outputs) that can be reshaped into a low-resolution version of our output image.

In [8]:
# define model
model = Sequential()

# define input shape, output enough activations for 128 versions of a 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))

# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))

The upsampled feature maps can be interpreted and filled in with hopefully useful detail by a Conv2D layer, which has a single feature map as output to create the single image we require.

In [9]:
# double input from 128 5x5 to 1 10x10 feature map
model.add(UpSampling2D())

# fill in detail in the upsampled feature maps
model.add(Conv2D(1, (3,3), padding='same'))

# summarize model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3200)              323200    
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


We can see that the Dense layer outputs 3,200 activations that are then reshaped into 128 feature maps with the shape 5×5.

The widths and heights are doubled to 10×10 by the UpSampling2D layer, resulting in a feature map with quadruple the area.

Finally, the Conv2D processes these feature maps and adds in detail, outputting a single 10×10 image.

## Worked Example Using the Conv2DTranspose Layer

In [10]:
# example of using the transpose convolutional layer
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2DTranspose

In [11]:
# define input data
X = asarray([[1, 2],
 [3, 4]])

# show input data for context
print(X)

[[1 2]
 [3 4]]


In [12]:
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

The model we define has only the Conv2DTranspose layer, which takes 2×2 grayscale images as input directly and outputs the result of the operation. It both upsamples and performs a convolution, so we must specify both the number of filters and the size of the filters.

Additionally, we must specify a stride of (2,2) because the upsampling is achieved by the stride behavior of the convolution on the input, and doing so has the effect of spacing out the input. Specifically, rows and columns of 0.0 values are inserted to achieve the desired stride.

In [13]:
# define model
model = Sequential()

# one filter, with a 1×1 kernel and a stride of 2×2 so that the 2×2 input image is upsampled to 4×4.
model.add(Conv2DTranspose(1, (1,1), strides=(2,2), input_shape=(2, 2, 1)))

# summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 4, 4, 1)           2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [14]:
# define weights that they do nothing
weights = [asarray([[[[1]]]]), asarray([0])]

# store the weights in the model
model.set_weights(weights)

In [15]:
# make a prediction with the model
yhat = model.predict(X)

# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))

# summarize output
print(yhat)

[[1. 0. 2. 0.]
 [0. 0. 0. 0.]
 [3. 0. 4. 0.]
 [0. 0. 0. 0.]]


*Unlike the UpSampling2D layer, the Conv2DTranspose will learn during training &  will attempt to fill in detail as part of the upsampling process.

## Simple Generator Model With the Conv2DTranspose Layer

The Conv2DTranspose is more complex than the UpSampling2D layer, but it is also effective when used in GAN models, specifically the generator model.

In this example, our little GAN generator model must produce a 10×10 image and take a 100-element vector from the latent space as input.

In [16]:
# example of using transpose conv in a simple generator model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.layers import Conv2D

In [17]:
# define model
model = Sequential()

# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))

# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))

Here, the 5×5 feature maps can be upsampled to a 10×10 feature map. We will use a 3×3 kernel size for the single filter, which will result in a slightly larger than doubled width and height in the output feature map (11×11).

In [18]:
# double input from 128 5x5 to 1 10x10 feature map (set padding to 'same' to ensure these output dimensions)
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same'))

# summarize model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape_1 (Reshape)          (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________
